In [1]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from numpy import load
from torch.utils.data import Dataset
from sklearn import preprocessing

In [2]:
# load array
Data = load('Data_09.npy')
# print the array

In [3]:
Data2=preprocessing.normalize(Data,axis=0)

In [4]:
Data=Data2

In [5]:
Data.shape

(174720, 7)

In [6]:
Data

array([[ 0.00089726,  0.00151341,  0.00125854, ...,  0.00017159,
         0.00058626,  0.        ],
       [ 0.00089726,  0.00199507,  0.00204135, ...,  0.00021997,
         0.00067893,  0.        ],
       [ 0.00089726,  0.0034888 ,  0.00313331, ...,  0.00027814,
         0.00080666,  0.        ],
       ...,
       [ 0.00041572, -0.00180586,  0.00066089, ...,  0.00021664,
         0.00030958,  0.        ],
       [ 0.00041572, -0.00172909,  0.00066089, ...,  0.00021451,
         0.00029199,  0.        ],
       [ 0.00041572, -0.00190667,  0.00066089, ...,  0.00021679,
         0.00027666,  0.        ]])

In [7]:
Xdata=[]
Ydata=[]
Xtrain_data=[]
Ytrain_data=[]
Xtest_data=[]
Ytest_data=[]



for idx in range(len(Data)//39):
    if(np.isinf(Data[39*idx:39*(idx+1)][:,:].tolist()).any()):
        print(np.isinf(Data[39*idx:39*(idx+1)][:,:].tolist()).any())
        continue
    if idx< (len(Data)//39)*0.9:
        Xtrain_data.append(Data[39*idx:39*(idx+1)][:,:].tolist())
        Ytrain_data.append(Data[39*idx:39*(idx+1)][:,1].tolist())
    else:
        Xtest_data.append(Data[39*idx:39*(idx+1)][:,:].tolist())
        Ytest_data.append(Data[39*idx:39*(idx+1)][:,1].tolist())
Xtrain_data=np.vstack(Xtrain_data)
Ytrain_data=np.vstack(Ytrain_data)
Xtest_data=np.vstack(Xtest_data)
Ytest_data=np.vstack(Ytest_data)

Ytrain_data=Ytrain_data[1:]>Ytrain_data[:-1]
Ytrain_data[38::39]=2
Xtrain_data=torch.FloatTensor(Xtrain_data)
Ytrain_data=torch.LongTensor(Ytrain_data)
Ytrain_data=Ytrain_data.view(-1)


Ytest_data=Ytest_data[1:]>Ytest_data[:-1]
Ytest_data[38::39]=2

Xtest_data=np.vstack(Xtest_data)
Ytest_data=np.vstack(Ytest_data)
Xtest_data=torch.FloatTensor(Xtest_data)
Ytest_data=torch.LongTensor(Ytest_data)
Ytest_data=Ytest_data.view(-1)

In [8]:
Ytrain_data=Ytrain_data[:,1:]>Ytrain_data[:,:-1]
Ytrain_data=torch.LongTensor(Ytrain_data)
Ytrain_data=torch.cat((torch.zeros(Ytrain_data.shape[0],1)+2,Ytrain_data),axis=1)
Ytrain_data=Ytrain_data.long()
Ytrain_data=Ytrain_data.view(-1)

Xtrain_data=torch.FloatTensor(Xtrain_data)


Ytest_data=Ytest_data[:,1:]>Ytest_data[:,:-1]
Ytest_data=torch.LongTensor(Ytest_data)
Ytest_data=torch.cat((torch.zeros(Ytest_data.shape[0],1)+2,Ytest_data),axis=1)

Xtest_data=np.vstack(Xtest_data)
Ytest_data=np.vstack(Ytest_data)
Xtest_data=torch.FloatTensor(Xtest_data)
Ytest_data=torch.LongTensor(Ytest_data)
Ytest_data=Ytest_data.view(-1)

Ytrain_data=2*(Ytrain_data>0).long()+(Ytrain_data==0).long()
Ytest_data=2*(Ytest_data>0).long()+(Ytest_data==0).long()

Ydata=(torch.stack([Ydata>0,Ydata==0, Ydata<0],axis=0)).float()

In [9]:
Ytrain_data=Ytrain_data.T
Ytest_data=Ytest_data.T

from torch.utils.data import DataLoader, random_split
Xtrain_data,Xtest_data=random_split(Xdata, [157700-15770, 15770])
Ytrain_data,Ytest_data=random_split(Ydata, [157700-15770, 15770])

Ytrain_data=torch.rand(Ytrain_data.shape)*3
Ytrain_data=Ytrain_data.long()

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz,bptt):
    # 데이터셋을 bsz 파트들로 나눕니다.
    if(data.size(0)%(bsz*bptt)!=0):
        data=data.view(-1,bptt,data.size(1)).transpose(0,1).contiguous()
        return data.to(device)
    nbatch = data.size(0) // bsz
    # 깔끔하게 나누어 떨어지지 않는 추가적인 부분(나머지들) 은 잘라냅니다.
    data = data.narrow(0, 0, nbatch * bsz)
    # 데이터에 대하여 bsz 배치들로 동등하게 나눕니다.
    data = data.view(bsz, -1,data.size(1)).transpose(0,1).contiguous()
    return data.to(device)

In [11]:
bptt = 39
def get_batch(source, i,bs):
    seq_len = min(bptt*bs, len(source)  - i)
    data = source[i:i+seq_len]
    if(seq_len!=bptt*bs):
        print(seq_len)
    target = source[i:i+seq_len].reshape(-1)
    return data, target

In [12]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
#         self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.src_tok_emb = nn.Linear(src_vocab_size, emb_size)
#         self.tgt_tok_emb = nn.Linear(3, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

#     def forward(self,
#                 src: Tensor,
#                 trg: Tensor,
#                 src_mask: Tensor,
#                 tgt_mask: Tensor,
#                 src_padding_mask: Tensor,
#                 tgt_padding_mask: Tensor,
#                 memory_key_padding_mask: Tensor):
    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                memory_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
#         tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
#         src_emb = self.positional_encoding(src)
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg).squeeze())
#         outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
#                                 src_padding_mask, tgt_padding_mask, memory_key_padding_mask)

        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask,memory_mask=memory_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

generate_square_subsequent_mask2(6,5)

In [13]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask
def generate_square_subsequent_mask2(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1)
    mask = mask.float().masked_fill(mask == 1, float('-inf')).masked_fill(mask == 0, float(0.0))
    return mask
def generate_square_subsequent_mask3(sz):
    mask = (torch.triu(torch.ones((sz, sz-1), device=DEVICE)) == 1).transpose(0,1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask
def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)
    src_mask = generate_square_subsequent_mask(src_seq_len)
    mmr_mask = generate_square_subsequent_mask3(src_seq_len)
    return src_mask, tgt_mask ,mmr_mask

In [14]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = Xtrain_data.shape[1]
TGT_VOCAB_SIZE = 3
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 10
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(transformer.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-9)

In [15]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    losses = 0
    total=0
    correct=0
    correct1=0
    tot1=0
    correct2=0
    tot2=0
    correct0=0
    tot0=0
    i=0
    while(i<Xtest_data.size(0)):
#    for batch, i in enumerate(range(0, Xtrain_data.size(0) - 1, BATCH_SIZE*bptt)):
        data, org_targets = get_batch(Xtrain_data, i,BATCH_SIZE)
        if (data.isnan().any() or data.isinf().any()):
            print(data)
            continue
#         _,targets = get_batch(Ytrain_data,i)
        targets,_ = get_batch(Ytrain_data,i,BATCH_SIZE)

        targets=torch.unsqueeze(targets,1)
        src=batchify(data,BATCH_SIZE,bptt)
        tgt=batchify(targets,BATCH_SIZE,bptt)
        src_input=src[:]
        tgt_input = tgt[:-1]


        src_mask, tgt_mask,mmr_mask = create_mask(src_input, tgt_input)
#        print(tgt_mask)
#        break
        logits = model(src_input, tgt_input, src_mask, tgt_mask,mmr_mask)
        optimizer.zero_grad()

        tgt_out = tgt[1:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
        if(loss.isnan()):
            print('NNNNN')
            print(loss)
        _,predicted=torch.max(logits,-1)
        correct += (tgt_out.squeeze() == predicted).sum().item()
        total+=len(predicted)*BATCH_SIZE
        tot0+=(0== tgt_out.squeeze()).sum().item()
        tot1+=(1== tgt_out.squeeze()).sum().item()
        tot2+=(2== tgt_out.squeeze()).sum().item()
        correct0+=((0== predicted) &(0==tgt_out.squeeze())).sum().item()
        correct1+=((1== predicted)&(1==tgt_out.squeeze())).sum().item()
        correct2+=((2== predicted)&(2==tgt_out.squeeze())).sum().item()
        
        i+=targets.size()[0]
    print(total)
    print(correct)
    print("TrainAcc:",correct/total*100)            
    print("TrainAcc0:",correct0,tot0*100,correct0,tot0)     
    print("TrainAcc1:",correct1,tot1*100,correct1,tot1)     
    print("TrainAcc2:",correct2,tot2*100,correct2,tot2)  
    return losses / Xtrain_data.size(0)

In [16]:
def evaluate(model):
    model.eval()
    losses = 0
    total=0
    correct=0
    correct1=0
    tot1=0
    correct2=0
    tot2=0
    correct0=0
    tot0=0
    with torch.no_grad():
        i=0
        while(i<Xtest_data.size(0)):
#        for i in range(0, Xtest_data.size(0) , bptt):

            data, org_targets = get_batch(Xtest_data, i,BATCH_SIZE)
            if (data.isnan().any() or data.isinf().any()):
                continue
            targets,_ = get_batch(Ytest_data,i,BATCH_SIZE)
            targets=torch.unsqueeze(targets,1)
            
#            print(data.size(),i)
            src=batchify(data,BATCH_SIZE,bptt)
            tgt=batchify(targets,BATCH_SIZE,bptt)

            src_input=src[:]

            
            tgt_input = tgt[:-1]

            src_mask, tgt_mask,mmr_mask = create_mask(src_input, tgt_input)

            logits = model(src_input, tgt_input, src_mask, tgt_mask,mmr_mask)

            tgt_out = tgt[1:]

            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
            
            
 #           if(loss.isnan()):
 #               print('NNNNN')
 #               print(loss)
#                print(src,tgt_input)
#                break
            losses += loss.item()
            _,predicted=torch.max(logits,-1)

            correct += (tgt_out.squeeze() == predicted).sum().item()
            total+=len(predicted)*BATCH_SIZE
            tot0+=(0== tgt_out.squeeze()).sum().item()
            tot1+=(1== tgt_out.squeeze()).sum().item()
            tot2+=(2== tgt_out.squeeze()).sum().item()
            if tot2!=0:
                print(i//39,tgt_out.shape)
                break
            correct0+=((0== predicted) &(0==tgt_out.squeeze())).sum().item()
            correct1+=((1== predicted)&(1==tgt_out.squeeze())).sum().item()
            correct2+=((2== predicted)&(2==tgt_out.squeeze())).sum().item()

            i+=targets.size()[0]
            
    print(total)
    print(correct)
    print("Acc:",correct/total*100)            
    print("Acc0:",correct0,tot0*100,correct0,tot0)     
    print("Acc1:",correct1,tot1*100,correct1,tot1)     
    print("Acc2:",correct2,tot2*100,correct2,tot2)     
    return losses / Xtest_data.size(0)

In [17]:
evaluate(transformer)

312
312
17100
8961
Acc: 52.40350877192983
Acc0: 8951 913900 8951 9139
Acc1: 10 788500 10 7885
Acc2: 0 0 0 0


0.0021437922110542273

In [18]:
torch.cuda.is_available()

True

In [19]:
from timeit import default_timer as timer
NUM_EPOCHS = 1000
best_val_loss=100000000
for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    print("Train_Loss:")
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    print("Val_Loss:")
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = transformer
PATH='best_model_seq_Trans_Midprice'
torch.save(best_model.state_dict(), PATH)


Train_Loss:
17100
8895
TrainAcc: 52.017543859649116
TrainAcc0: 5623 937100 5623 9371
TrainAcc1: 3272 772900 3272 7729
TrainAcc2: 0 0 0 0
Val_Loss:
312
312
17100
9240
Acc: 54.03508771929825
Acc0: 4037 913900 4037 9139
Acc1: 5203 788500 5203 7885
Acc2: 0 0 0 0
Epoch: 1, Train loss: 0.000, Val loss: 0.002, Epoch time = 1.515s
Train_Loss:
17100
9161
TrainAcc: 53.57309941520468
TrainAcc0: 6134 937100 6134 9371
TrainAcc1: 3027 772900 3027 7729
TrainAcc2: 0 0 0 0
Val_Loss:
312
312
17100
9558
Acc: 55.89473684210526
Acc0: 6171 913900 6171 9139
Acc1: 3387 788500 3387 7885
Acc2: 0 0 0 0
Epoch: 2, Train loss: 0.000, Val loss: 0.002, Epoch time = 1.438s
Train_Loss:
17100
9012
TrainAcc: 52.70175438596492
TrainAcc0: 6133 937100 6133 9371
TrainAcc1: 2879 772900 2879 7729
TrainAcc2: 0 0 0 0
Val_Loss:
312
312
17100
9525
Acc: 55.70175438596491
Acc0: 5540 913900 5540 9139
Acc1: 3985 788500 3985 7885
Acc2: 0 0 0 0
Epoch: 3, Train loss: 0.000, Val loss: 0.002, Epoch time = 1.531s
Train_Loss:
17100
9179
Trai

17100
9151
Acc: 53.514619883040936
Acc0: 3379 913900 3379 9139
Acc1: 5772 788500 5772 7885
Acc2: 0 0 0 0
Epoch: 26, Train loss: 0.000, Val loss: 0.002, Epoch time = 1.483s
Train_Loss:
17100
9423
TrainAcc: 55.10526315789473
TrainAcc0: 7043 937100 7043 9371
TrainAcc1: 2380 772900 2380 7729
TrainAcc2: 0 0 0 0
Val_Loss:
312
312
17100
9583
Acc: 56.04093567251462
Acc0: 4935 913900 4935 9139
Acc1: 4648 788500 4648 7885
Acc2: 0 0 0 0
Epoch: 27, Train loss: 0.000, Val loss: 0.002, Epoch time = 1.429s
Train_Loss:
17100
9469
TrainAcc: 55.37426900584796
TrainAcc0: 6991 937100 6991 9371
TrainAcc1: 2478 772900 2478 7729
TrainAcc2: 0 0 0 0
Val_Loss:
312
312
17100
9522
Acc: 55.684210526315795
Acc0: 4685 913900 4685 9139
Acc1: 4837 788500 4837 7885
Acc2: 0 0 0 0
Epoch: 28, Train loss: 0.000, Val loss: 0.002, Epoch time = 1.442s
Train_Loss:
17100
9420
TrainAcc: 55.08771929824562
TrainAcc0: 7076 937100 7076 9371
TrainAcc1: 2344 772900 2344 7729
TrainAcc2: 0 0 0 0
Val_Loss:
312
312
17100
9610
Acc: 56.1988

17100
9549
TrainAcc: 55.84210526315789
TrainAcc0: 7129 937100 7129 9371
TrainAcc1: 2420 772900 2420 7729
TrainAcc2: 0 0 0 0
Val_Loss:
312
312
17100
9576
Acc: 56.00000000000001
Acc0: 6266 913900 6266 9139
Acc1: 3310 788500 3310 7885
Acc2: 0 0 0 0
Epoch: 52, Train loss: 0.000, Val loss: 0.002, Epoch time = 1.467s
Train_Loss:
17100
9568
TrainAcc: 55.95321637426901
TrainAcc0: 7216 937100 7216 9371
TrainAcc1: 2352 772900 2352 7729
TrainAcc2: 0 0 0 0
Val_Loss:
312
312
17100
9624
Acc: 56.280701754385966
Acc0: 6000 913900 6000 9139
Acc1: 3624 788500 3624 7885
Acc2: 0 0 0 0
Epoch: 53, Train loss: 0.000, Val loss: 0.002, Epoch time = 1.494s
Train_Loss:
17100
9634
TrainAcc: 56.33918128654971
TrainAcc0: 7051 937100 7051 9371
TrainAcc1: 2583 772900 2583 7729
TrainAcc2: 0 0 0 0
Val_Loss:
312
312
17100
9603
Acc: 56.15789473684211
Acc0: 5857 913900 5857 9139
Acc1: 3746 788500 3746 7885
Acc2: 0 0 0 0
Epoch: 54, Train loss: 0.000, Val loss: 0.002, Epoch time = 1.455s
Train_Loss:
17100
9581
TrainAcc: 56.

312
312
17100
9430
Acc: 55.146198830409354
Acc0: 7405 913900 7405 9139
Acc1: 2025 788500 2025 7885
Acc2: 0 0 0 0
Epoch: 77, Train loss: 0.000, Val loss: 0.002, Epoch time = 1.467s
Train_Loss:
17100
9528
TrainAcc: 55.719298245614034
TrainAcc0: 7184 937100 7184 9371
TrainAcc1: 2344 772900 2344 7729
TrainAcc2: 0 0 0 0
Val_Loss:
312
312
17100
9623
Acc: 56.27485380116959
Acc0: 6303 913900 6303 9139
Acc1: 3320 788500 3320 7885
Acc2: 0 0 0 0
Epoch: 78, Train loss: 0.000, Val loss: 0.002, Epoch time = 1.437s
Train_Loss:
17100
9593
TrainAcc: 56.09941520467836
TrainAcc0: 7067 937100 7067 9371
TrainAcc1: 2526 772900 2526 7729
TrainAcc2: 0 0 0 0
Val_Loss:
312
312
17100
9583
Acc: 56.04093567251462
Acc0: 6262 913900 6262 9139
Acc1: 3321 788500 3321 7885
Acc2: 0 0 0 0
Epoch: 79, Train loss: 0.000, Val loss: 0.002, Epoch time = 1.465s
Train_Loss:
17100
9651
TrainAcc: 56.43859649122807
TrainAcc0: 7011 937100 7011 9371
TrainAcc1: 2640 772900 2640 7729
TrainAcc2: 0 0 0 0
Val_Loss:
312
312
17100
9558
Acc:

17100
9738
TrainAcc: 56.94736842105264
TrainAcc0: 7134 937100 7134 9371
TrainAcc1: 2604 772900 2604 7729
TrainAcc2: 0 0 0 0
Val_Loss:
312
312
17100
9594
Acc: 56.10526315789473
Acc0: 6524 913900 6524 9139
Acc1: 3070 788500 3070 7885
Acc2: 0 0 0 0
Epoch: 103, Train loss: 0.000, Val loss: 0.002, Epoch time = 1.489s
Train_Loss:
17100
9788
TrainAcc: 57.239766081871345
TrainAcc0: 7176 937100 7176 9371
TrainAcc1: 2612 772900 2612 7729
TrainAcc2: 0 0 0 0
Val_Loss:
312
312
17100
9524
Acc: 55.695906432748544
Acc0: 6860 913900 6860 9139
Acc1: 2664 788500 2664 7885
Acc2: 0 0 0 0
Epoch: 104, Train loss: 0.000, Val loss: 0.002, Epoch time = 1.498s
Train_Loss:
17100
9724
TrainAcc: 56.865497076023395
TrainAcc0: 7130 937100 7130 9371
TrainAcc1: 2594 772900 2594 7729
TrainAcc2: 0 0 0 0
Val_Loss:
312
312
17100
9559
Acc: 55.90058479532164
Acc0: 6549 913900 6549 9139
Acc1: 3010 788500 3010 7885
Acc2: 0 0 0 0
Epoch: 105, Train loss: 0.000, Val loss: 0.002, Epoch time = 1.448s
Train_Loss:
17100
9700
TrainAcc

KeyboardInterrupt: 

In [ ]:
best_val_loss
evaluate(best_model)